In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/GrandTourjc.png"

In [ ]:
s3.upload_file(
    Filename="resources/GrandTourjc.png",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

In [ ]:
recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
display(recognizeCelebritiesResponse)

In [ ]:
def drawBoundingBoxes (sourceImage, boxes, outputImage):
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    imageLocation = 'resources/'+os.path.basename(sourceImage)

    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(outputImage,format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
celebrities = recognizeCelebritiesResponse['CelebrityFaces']
for celebrity in celebrities:
    boxes.append ((celebrity['Name'], celebrity['Face']['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes, 'resources/GrandTourjc2.png')

# Reconociendo celebridades en video

In [ ]:
bucketName = "temporal-bucket-max"
videoName = "media/GrandTour720.mp4"

In [ ]:
s3.upload_file(
    Filename="resources/GrandTour720.mp4",
    Bucket=bucketName,
    Key=videoName,
)

In [ ]:
mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
startCelebrityRekognition = rekognition.start_celebrity_recognition(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

celebrityJobId = startCelebrityRekognition['JobId']
display("Job Id: {0}".format(celebrityJobId))

In [ ]:
getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP'
)

while(getCelebrityRecognition['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP')
    
display(getCelebrityRecognition['JobStatus'])

In [ ]:
theCelebs = {}

strDetail = "Celebrites detected in video<br>=======================================<br>"
strOverall = "Celebrities in the overall video:<br>=======================================<br>"

for celebrity in getCelebrityRecognition['Celebrities']:
    if 'Celebrity' in celebrity :
        cconfidence = celebrity["Celebrity"]["Confidence"]
        if(cconfidence > 95):
            ts = celebrity ["Timestamp"]
            cname = celebrity["Celebrity"]["Name"]
            strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, cname, round(cconfidence,2))
            if not cname in theCelebs:
                theCelebs[cname] = cname
            
for theCeleb in theCelebs:
    strOverall = strOverall + "Name: {}<br>".format(theCeleb)

display(HTML(strOverall))